In [1]:
import pandas as pd
import json
import ast
from tabulate import tabulate

## Funcion 02
#### def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.
Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

ARMAMOS LOS 3 DF EN BASE A LOS REQUERIMENTOS, Y LUEGO LOS UNIMOS PARA DEJAR SOLO LOS DATOS NECESARIOS PARA LA FUNCION EN 1 SOLO DF

REVIEWS

In [2]:
dfreviews= pd.read_csv('df_Analisis_Sentimiento.csv')

In [3]:
dfreviews.head()

,user_id,item_id,sentiment_analysis
0,76561197970982479,1250.0,2
1,76561197970982479,22200.0,2
2,76561197970982479,43110.0,2
3,js41637,251610.0,2
4,js41637,227300.0,0


In [4]:
dfreviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             59333 non-null  object 
 1   item_id             59305 non-null  float64
 2   sentiment_analysis  59333 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.4+ MB


GAMES

In [5]:
row2 = []
with open('Datasets/steam_games.json', 'r') as f:
        for line in f.readlines():
            data = json.loads(line)
            row2.append(data)

dfgame = pd.DataFrame(row2)
# limpio los valores nulos y los duplicados, y selecciono solo las columnas 'id' y 'price' para el DataFrame final dfgame.
dfgame.dropna(how='all', inplace=True)
dfgame.dropna(subset=['id', 'price'], inplace=True)
dfgame.drop_duplicates(subset='id', keep='first', inplace=True)
dfgame = dfgame[['id', 'price']]
dfgame.head()

,id,price
88310,761140,4.99
88311,643980,Free To Play
88312,670290,Free to Play
88313,767400,0.99
88314,773570,2.99


In [6]:
dfgame.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30755 entries, 88310 to 120444
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      30755 non-null  object
 1   price   30755 non-null  object
dtypes: object(2)
memory usage: 720.8+ KB


ITEMS

In [7]:
row3 = []
with open('Datasets/users_items.json', 'rt',encoding= 'utf-8') as f:
    for line in f.readlines():
        row3.append(ast.literal_eval(line))
dfitem = pd.DataFrame(row3)
dfitem.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [8]:
# Desanidar la columna 'items'
dfitem_exploded = dfitem.explode('items')

# Crear columnas 'item_id' y 'playtime_forever' a partir de 'items'
dfitem_exploded['item_id'] = dfitem_exploded['items'].apply(lambda x: x.get('item_id') if isinstance(x, dict) else '')
dfitem_exploded['playtime_forever'] = dfitem_exploded['items'].apply(lambda x: x.get('playtime_forever') if isinstance(x, dict) else '')

# Eliminar la columna 'items' original
dfitem_exploded.drop('items', axis=1, inplace=True)

In [9]:
# df con las columnas requeridas
dfitem_exploded=dfitem_exploded[['user_id', 'item_id', 'playtime_forever']]

In [10]:
dfitem_exploded.head()

,user_id,item_id,playtime_forever
0,76561197970982479,10,6
0,76561197970982479,20,0
0,76561197970982479,30,7
0,76561197970982479,40,0
0,76561197970982479,50,0


In [11]:
dfitem_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5170015 entries, 0 to 88309
Data columns (total 3 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   item_id           object
 2   playtime_forever  object
dtypes: object(3)
memory usage: 157.8+ MB


LIMPIAMOS PARA HACER LOS MERGE

In [12]:
# Limpiar dfreviews
dfreviews = dfreviews.dropna(subset=['user_id', 'item_id'])
dfreviews = dfreviews[dfreviews['user_id'].notna()]
dfreviews = dfreviews[dfreviews['item_id'].notna()]

# Limpiar dfgame
dfgame = dfgame.dropna(subset=['id', 'price'])
dfgame = dfgame[dfgame['id'].notna()]
dfgame = dfgame[dfgame['price'].notna()]

# Limpiar dfitem_exploded
dfitem_exploded = dfitem_exploded.dropna(subset=['user_id', 'item_id', 'playtime_forever'])
dfitem_exploded = dfitem_exploded[dfitem_exploded['user_id'].notna()]
dfitem_exploded = dfitem_exploded[dfitem_exploded['item_id'].notna()]
dfitem_exploded = dfitem_exploded[dfitem_exploded['playtime_forever'].notna()]

# Convertir 'item_id' en dfitem_exploded a int64 para poder fusionar con 'id' en dfgame
dfitem_exploded['item_id'] = pd.to_numeric(dfitem_exploded['item_id'], errors='coerce')

# Convertir 'id' en dfgame a object para que coincida con 'item_id'
dfgame['id'] = pd.to_numeric(dfgame['id'], errors='coerce')

# Fusionar dfitem_exploded y dfgame
df_merged = pd.merge(dfitem_exploded, dfgame, left_on='item_id', right_on='id', how='left')

# Agregar la columna 'recommend' del dfreviews a df_merged
df_merged = pd.merge(df_merged, dfreviews[['user_id', 'item_id', 'sentiment_analysis']], 
                    left_on=['user_id', 'item_id'], right_on=['user_id', 'item_id'], how='left')

# Renombrar la columna 'sentiment_analysis' a 'recommend'
df_merged.rename(columns={'sentiment_analysis': 'recommend'}, inplace=True)





In [13]:
df_merged.head()

,user_id,item_id,playtime_forever,id,price,recommend
0,76561197970982479,10.0,6,10.0,9.99,NaN
1,76561197970982479,20.0,0,20.0,4.99,NaN
2,76561197970982479,30.0,7,30.0,4.99,NaN
3,76561197970982479,40.0,0,40.0,4.99,NaN
4,76561197970982479,50.0,0,50.0,4.99,NaN


In [14]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171439 entries, 0 to 5171438
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   item_id           float64
 2   playtime_forever  object 
 3   id                float64
 4   price             object 
 5   recommend         float64
dtypes: float64(3), object(3)
memory usage: 236.7+ MB


In [15]:
# Eliminar filas con playtime_forever igual a 0
df_merged = df_merged[df_merged['playtime_forever'] != 0]

In [16]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3303472 entries, 0 to 5171438
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   item_id           float64
 2   playtime_forever  object 
 3   id                float64
 4   price             object 
 5   recommend         float64
dtypes: float64(3), object(3)
memory usage: 176.4+ MB


Eliminamos 2 columnas que no hacen falta:

In [17]:
df_merged = df_merged.drop(['id'], axis=1)
df_merged = df_merged.drop(['playtime_forever'], axis=1)


In [18]:
df_merged.head()

,user_id,item_id,price,recommend
0,76561197970982479,10.0,9.99,NaN
2,76561197970982479,30.0,4.99,NaN
8,76561197970982479,300.0,9.99,NaN
9,76561197970982479,240.0,19.99,NaN
10,76561197970982479,3830.0,9.99,NaN


In [19]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3303472 entries, 0 to 5171438
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   user_id    object 
 1   item_id    float64
 2   price      object 
 3   recommend  float64
dtypes: float64(2), object(2)
memory usage: 126.0+ MB


GENERAMOS TABLA NECESARIA PARA FUNCION 02

In [20]:
# Convertir el tipo de datos de 'item_id' y 'price' a numérico
df_merged['item_id'] = pd.to_numeric(df_merged['item_id'])
df_merged['price'] = pd.to_numeric(df_merged['price'], errors='coerce')

# Calcular el gasto total por usuario
df_merged['total_spent'] = df_merged['item_id'] * df_merged['price']

# Calcular el total de items jugados por usuario
df_merged_grouped = df_merged.groupby('user_id').agg({'total_spent': 'sum', 'item_id': 'count', 'recommend': lambda x: (x == 2).sum() / len(x) * 100})

# Renombrar las columnas
df_merged_grouped.rename(columns={'item_id': 'total_items_played', 'recommend': 'percent_recommended'}, inplace=True)

In [21]:
df_merged_grouped.head()

,total_spent,total_items_played,percent_recommended
user_id,,,
--000--,19913801.7,41,0.000000
--ace--,15359533.9,25,4.000000
--ionex--,8288080.1,22,9.090909
-2SV-vuLB-Kg,62877422.0,43,9.302326
-404PageNotFound-,199784894.4,110,0.000000


In [25]:
df_merged_grouped.reset_index(inplace=True)

In [27]:
df_merged_grouped.head()


,user_id,total_spent,total_items_played,percent_recommended
0,--000--,19913801.7,41,0.000000
1,--ace--,15359533.9,25,4.000000
2,--ionex--,8288080.1,22,9.090909
3,-2SV-vuLB-Kg,62877422.0,43,9.302326
4,-404PageNotFound-,199784894.4,110,0.000000


In [26]:
df_merged_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85117 entries, 0 to 85116
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              85117 non-null  object 
 1   total_spent          85117 non-null  float64
 2   total_items_played   85117 non-null  int64  
 3   percent_recommended  85117 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 2.6+ MB


In [29]:
df_merged_grouped['user_id'] = df_merged_grouped['user_id'].astype(str)

In [30]:
df_merged_grouped.to_csv('df_Funcion2.csv', index=False)

Funcion 02 :

In [34]:
def userdata(user_id: str):
    import pandas as pd
    df_merged_grouped = pd.read_csv('df_Funcion2.csv')

    if user_id in df_merged_grouped['user_id'].values:
        row = df_merged_grouped[df_merged_grouped['user_id'] == user_id]
        dinero_gastado = row['total_spent'].values[0]
        recomendacion = row['percent_recommended'].values[0]
        cantidad_items = row['total_items_played'].values[0]

        return {
            "Usuario": user_id,
            "Dinero gastado": f"{dinero_gastado:.2f} USD",
            "porcentaje de recomendación": f"{recomendacion:.2f}%",
            "Cantidad de items": cantidad_items
        }
    else:
        return f"El usuario con ID {user_id} no se encuentra en la base de datos."

In [35]:
userdata("76561197970982479")


{'Usuario': '76561197970982479',
 'Dinero gastado': '367518968.40 USD',
 'porcentaje de recomendación': '1.52%',
 'Cantidad de items': 198}